In [1]:
import json
import csv

import numpy as np
import matplotlib.pyplot as plt
import torch
import copy
import math
import pickle
import sys
sys.path.append("../../")

import implicitmodules.torch as dm


a = 1.

shape0 = torch.tensor([[-a, -a], [a, -a], [a, a], [-a, a]]) -torch.tensor([2., 0.])
shape0.requires_grad_()
a=1.
shape1 = torch.tensor([[-a, -a], [a, -a], [a, a], [-a, a]]) + torch.tensor([2., 0.])
shape1.requires_grad_()

a = math.sqrt(2)
target0 = torch.tensor([[0., -a], [a, 0.], [0., a], [-a, 0.]]) -torch.tensor([2., 0.])
target0.requires_grad_()

a=1.5
target1 = torch.tensor([[-a, -a], [a, -a], [a, a], [-a, a]]) + torch.tensor([2., 0.])
target1.requires_grad_()

scale_scaling = 2.
scaling = dm.DeformationModules.LocalScaling(2, scale_scaling, gd=torch.tensor([[2., 0.]], requires_grad=True))
scale_rotation = 2.
rotation = dm.DeformationModules.LocalRotation(2, scale_rotation, gd=torch.tensor([[-2., 0.]], requires_grad=True))

boundary0 = dm.MultiShape.Boundary(shape0)
boundary1 = dm.MultiShape.Boundary(shape1)

source_deformable0 = dm.Models.DeformablePoints(shape0)
source_deformable1 = dm.Models.DeformablePoints(shape1)
target_deformable0 = dm.Models.DeformablePoints(target0)
target_deformable1 = dm.Models.DeformablePoints(target1)


plt.plot(shape0[:,0].detach(), shape0[:,1].detach(), '-b')
plt.plot(shape1[:,0].detach(), shape1[:,1].detach(), '-b')
plt.plot(target0[:,0].detach(), target0[:,1].detach(), '-r')
plt.plot(target1[:,0].detach(), target1[:,1].detach(), '-r')
plt.axis('equal')
plt.show()


attach = dm.Attachment.EuclideanPointwiseDistanceAttachment()

shoot_solver = 'torch_euler'
shoot_it = 10

compound_man = dm.MultiShape.MultishapeCompoundManifold.MultishapeCompoundManifold([dm.Manifolds.CompoundManifold([source_deformable0.silent_module.manifold, rotation.manifold, boundary0.silent_module.manifold]), dm.Manifolds.CompoundManifold([source_deformable1.silent_module.manifold, scaling.manifold, boundary0.silent_module.manifold]), dm.Manifolds.CompoundManifold([boundary0.silent_module.manifold, boundary1.silent_module.manifold])])

#constraints = dm.Constraints.Constraints.ConstraintsPointIdentityBase([0, 0], [1, 0], compound_man)
constraints0 = dm.Constraints.Constraints.ConstraintsPointIdentityBackground(0, compound_man)
constraints1 = dm.Constraints.Constraints.ConstraintsPointIdentityBackground(1, compound_man)
constraints = dm.Constraints.Constraints.CompoundConstraints([constraints0, constraints1])
model = dm.Models.RegistrationModelMultishape([boundary0, boundary1], [source_deformable0, source_deformable1], [[rotation], [scaling]], [attach, attach], 0.5, constraints=constraints)

multishape = dm.MultiShape.MultiShape.MultiShapeModules(model.modules, 0.5)

<Figure size 640x480 with 1 Axes>

In [2]:
intermediates = {}
model.evaluate([target_deformable0, target_deformable1], shoot_solver, shoot_it)

print constraints
tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]], grad_fn=<CatBackward>)
print constraints done
print constraints
tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]], grad_fn=<CatBackward>)
print constraints done
print constraints
tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]], grad_fn=<CatBackward>)
print constraints done
print constraints
tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
   

{'attach': 7.15799617767334, 'deformation': 0.0}